In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
import pandas as pd
from scipy.io import loadmat

import optuna


M = 100

In [7]:
###
def GetVanillaEncoders(DataDimension,LatentDim,opti_fun):
#Vanilla decoder and decoder
    vanillaEncoder = tf.keras.models.Sequential([
    tf.keras.Input(shape=(DataDimension,)),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(LatentDim,activation="sigmoid")])
    
    
    vanillaDecoder = tf.keras.models.Sequential([
    tf.keras.Input(shape=(LatentDim,)),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(DataDimension,activation="sigmoid")])

    vanillaAutoEncoder = tf.keras.models.Sequential([vanillaEncoder,vanillaDecoder])
    #Training
    vanillaAutoEncoder.compile(optimizer=opti_fun,loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM))
    return vanillaEncoder,vanillaDecoder,vanillaAutoEncoder


In [3]:
def GetBayesEncoders(DataDimension,LatentDim,IntermediateLayerDimension, opti_fun):
    ###
    #Bayes auto encoder

    ###
    #Decoder model, with Bernouilli
    decoder = tf.keras.models.Sequential([
      tf.keras.Input(shape=(LatentDim,)),
      tf.keras.layers.Dense(IntermediateLayerDimension, activation=tf.math.tanh),
      tf.keras.layers.Dense(DataDimension,activation = "sigmoid")
      ])
            


    #Coder model, can't use sequential easily because of two values are returned : mu and sigma
    x = tf.keras.Input(shape=(DataDimension,))
    h = tf.keras.layers.Dense(IntermediateLayerDimension, activation=tf.math.tanh)(x)
    zMu = tf.keras.layers.Dense(LatentDim)(h)
    zLogVar = tf.keras.layers.Dense(LatentDim)(h)
    zSigma = tf.math.exp(.5*zLogVar)
    ###
    
    #Noise Layer
    eps = tf.keras.layers.Lambda(lambda t: tf.keras.backend.random_normal(shape=(M, LatentDim)))(zLogVar)
    
    z = zMu + zSigma*eps
    decodedX = decoder(z)


    vae = tf.keras.Model(inputs=[x], outputs=decodedX)

    #Adding KL_divergence
    KLDivergence = 0.5 * tf.reduce_sum(tf.square(zMu) + tf.square(zSigma) - tf.math.log(1e-8 + tf.square(zSigma)) - 1, 1)
    KLDivergence = tf.reduce_mean(KLDivergence)
    vae.add_loss(KLDivergence)

    vae.compile(optimizer = opti_fun,
            loss=tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM))
    
    encoder = tf.keras.Model(x, zMu)
    
    return encoder,decoder,vae

In [4]:
def LoadMNIST():
    (xTrain, yTrain), (xTest, yTest) = tf.keras.datasets.mnist.load_data()
    DataDimension = xTrain.shape[1]*xTrain.shape[2]
    xTrain = xTrain.reshape(-1, DataDimension) / 255.
    xTest = xTest.reshape(-1, DataDimension) / 255.   
    return xTrain,yTrain,xTest,yTest,DataDimension


In [5]:
def LoadFrey():
    # reshape data for later convenience
    img_rows, img_cols = 28, 20
    ff = loadmat(r"C:\Users\bapti\Downloads\frey_rawface.mat", squeeze_me=True, struct_as_record=False)
    ff = ff["ff"].T.reshape((-1, img_rows, img_cols))

    np.random.seed(42)
    n_pixels = img_rows * img_cols
    X_train = ff[:1800]
    X_val = ff[1800:1900]
    X_train = X_train.astype('float32') / 255.
    X_val = X_val.astype('float32') / 255.
    X_train = X_train.reshape((len(X_train), n_pixels))
    X_val = X_val.reshape((len(X_val), n_pixels))
    return X_train,X_train,X_val,X_val,560

In [8]:
xTrain,yTrain,xTest,yTest,DataDimension = LoadMNIST()
def objective(trial):
    
    lr = trial.suggest_loguniform('lr', 1e-9, 1e-1)
    rho = trial.suggest_float("rho",0.1,1-1e-3)
    momentum = trial.suggest_float("momentum",0,1-1e-3)
    
    opti_fun = tf.keras.optimizers.RMSprop(learning_rate=lr, rho=rho, momentum=momentum, epsilon=1e-07, centered=False)
    LatentDim = 5
    IntermediateLayerDimension = 500
    encoder,decoder,autoEncoder = GetVanillaEncoders(DataDimension,IntermediateLayerDimension,opti_fun )
    
    epochs = 20

    history = autoEncoder.fit(xTrain,xTrain,shuffle=True,epochs=epochs,batch_size=M,validation_data=(xTest, xTest))
    
    return history.history["val_loss"][-1]

study2 = optuna.create_study()
study2.optimize(objective, n_trials=200)

study2.best_params

[I 2022-01-08 22:53:38,453] A new study created in memory with name: no-name-4f567cfd-a654-4928-bf78-cb5863024e2c


Epoch 1/20
600/600 [==============================] - 2s 3ms/step - loss: 24.9990 - val_loss: 18.5022
Epoch 2/20
600/600 [==============================] - 2s 3ms/step - loss: 17.3103 - val_loss: 16.2120
Epoch 3/20
600/600 [==============================] - 1s 2ms/step - loss: 16.0129 - val_loss: 15.3255
Epoch 4/20
600/600 [==============================] - 1s 2ms/step - loss: 15.5454 - val_loss: 15.0857
Epoch 5/20
600/600 [==============================] - 1s 2ms/step - loss: 15.2322 - val_loss: 15.6038
Epoch 6/20
600/600 [==============================] - 1s 2ms/step - loss: 14.9494 - val_loss: 14.5445
Epoch 7/20
600/600 [==============================] - 1s 2ms/step - loss: 14.7776 - val_loss: 14.5595
Epoch 8/20
600/600 [==============================] - 1s 2ms/step - loss: 14.5460 - val_loss: 14.3932
Epoch 9/20
600/600 [==============================] - 1s 2ms/step - loss: 14.4498 - val_loss: 14.5178
Epoch 10/20
600/600 [==============================] - 1s 2ms/step - loss: 14.3080

[I 2022-01-08 22:54:20,809] Trial 0 finished with value: 13.126771926879883 and parameters: {'lr': 0.015234480260706782, 'rho': 0.7157910562700234, 'momentum': 0.14379582618962453, 'nesterov': 1}. Best is trial 0 with value: 13.126771926879883.


Epoch 1/20
600/600 [==============================] - 2s 3ms/step - loss: 25.7395 - val_loss: 18.2162
Epoch 2/20
600/600 [==============================] - 1s 2ms/step - loss: 16.2016 - val_loss: 14.4275
Epoch 3/20
600/600 [==============================] - 1s 2ms/step - loss: 13.5214 - val_loss: 12.4971
Epoch 4/20
600/600 [==============================] - 1s 2ms/step - loss: 11.9921 - val_loss: 11.2267
Epoch 5/20
600/600 [==============================] - 1s 2ms/step - loss: 10.9256 - val_loss: 10.3708
Epoch 6/20
600/600 [==============================] - 1s 2ms/step - loss: 10.2597 - val_loss: 9.8626
Epoch 7/20
600/600 [==============================] - 1s 2ms/step - loss: 9.8275 - val_loss: 9.5006
Epoch 8/20
600/600 [==============================] - 1s 2ms/step - loss: 9.4900 - val_loss: 9.2057
Epoch 9/20
600/600 [==============================] - 1s 2ms/step - loss: 9.2078 - val_loss: 8.9543
Epoch 10/20
600/600 [==============================] - 1s 2ms/step - loss: 8.9702 - val_l

[I 2022-01-08 22:54:46,895] Trial 1 finished with value: 7.761109352111816 and parameters: {'lr': 2.8727448504517618e-05, 'rho': 0.41255732281535307, 'momentum': 0.6515434805224991, 'nesterov': 0}. Best is trial 1 with value: 7.761109352111816.


...

Epoch 1/20
600/600 [==============================] - 2s 4ms/step - loss: 12.6118 - val_loss: 8.2953
Epoch 2/20
600/600 [==============================] - 2s 3ms/step - loss: 7.8819 - val_loss: 7.4515
Epoch 3/20
600/600 [==============================] - 1s 2ms/step - loss: 7.3196 - val_loss: 7.1234
Epoch 4/20
600/600 [==============================] - 1s 2ms/step - loss: 7.0710 - val_loss: 6.9550
Epoch 5/20
600/600 [==============================] - 1s 2ms/step - loss: 6.9223 - val_loss: 6.8325
Epoch 6/20
600/600 [==============================] - 1s 2ms/step - loss: 6.8224 - val_loss: 6.7710
Epoch 7/20
600/600 [==============================] - 1s 2ms/step - loss: 6.7506 - val_loss: 6.6983
Epoch 8/20
600/600 [==============================] - 1s 2ms/step - loss: 6.6966 - val_loss: 6.6480
Epoch 9/20
600/600 [==============================] - 1s 2ms/step - loss: 6.6536 - val_loss: 6.6194
Epoch 10/20
600/600 [==============================] - 1s 2ms/step - loss: 6.6206 - val_loss: 6.595

[I 2022-01-09 00:41:02,857] Trial 198 finished with value: 6.470250606536865 and parameters: {'lr': 0.00013780902463791743, 'rho': 0.9165886687731356, 'momentum': 0.920763243848461, 'nesterov': 1}. Best is trial 183 with value: 6.4513068199157715.


Epoch 1/20
600/600 [==============================] - 1s 2ms/step - loss: 12.5281 - val_loss: 8.3633
Epoch 2/20
600/600 [==============================] - 1s 2ms/step - loss: 7.9641 - val_loss: 7.5503
Epoch 3/20
600/600 [==============================] - 1s 2ms/step - loss: 7.3993 - val_loss: 7.1698
Epoch 4/20
600/600 [==============================] - 1s 2ms/step - loss: 7.1217 - val_loss: 7.0157
Epoch 5/20
600/600 [==============================] - 1s 2ms/step - loss: 6.9629 - val_loss: 6.8694
Epoch 6/20
600/600 [==============================] - 1s 2ms/step - loss: 6.8547 - val_loss: 6.7792
Epoch 7/20
600/600 [==============================] - 1s 2ms/step - loss: 6.7794 - val_loss: 6.7271
Epoch 8/20
600/600 [==============================] - 1s 2ms/step - loss: 6.7207 - val_loss: 6.6637
Epoch 9/20
600/600 [==============================] - 1s 2ms/step - loss: 6.6755 - val_loss: 6.6390
Epoch 10/20
600/600 [==============================] - 1s 2ms/step - loss: 6.6387 - val_loss: 6.614

[I 2022-01-09 00:41:28,881] Trial 199 finished with value: 6.46063756942749 and parameters: {'lr': 0.0001468542190031063, 'rho': 0.9072478901850067, 'momentum': 0.9026351015387389, 'nesterov': 1}. Best is trial 183 with value: 6.4513068199157715.


{'lr': 4.550835231920952e-05,
 'rho': 0.8782246913655031,
 'momentum': 0.9669845443339055,
 'nesterov': 1}

In [ ]:



IntermediateLayerDimension = 500
encoder,decoder,autoEncoder = GetBayesEncoders(DataDimension,LatentDim,IntermediateLayerDimension)

epochs = 10

history = autoEncoder.fit(xTrain,xTrain,shuffle=True,epochs=epochs,batch_size=M,validation_data=(xTest, xTest))

In [ ]:
pd.DataFrame(history.history).plot() 
